In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from IPython.core.display import clear_output
from urllib.parse import quote
# import chromedriver_autoinstaller

In [2]:
def get_search_html(url):
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    return html_soup

In [24]:
def get_movie_url(movie, year):
    url = r"https://www.rottentomatoes.com/search?search=" + quote(movie)
    print(url)
    movie_url = None
    print((movie, year))
    
    try:
        soup = get_search_html(url)
        
        mov = soup.find('search-page-media-row', releaseyear= [y for y in range(year-1, year+2)])
        if mov is not None:
            movie_url = mov.find('a')['href']
        
    except NoSuchElementException:
        print("Movie not found!!")
    
    return movie_url
    

In [209]:
def get_rt_scores(movie, year):
    print(movie)
    movie_url = get_movie_url(movie, year)
    print(movie_url)
    a = np.nan
    c = np.nan

    if movie_url is not None:
        try:
            response = get(movie_url)
            html_soup = BeautifulSoup(response.text, 'html.parser')

            x = html_soup.find('score-board')

            a = x['audiencescore']
            c = x['tomatometerscore']
        except TypeError:
            errors.append(movie)
            print("TypeError raised")

        print("get_rt_scores\nMovie:{},critic:{},audience:{}".format(
            movie, c, a))

    return (pd.Series([c, a], index = ['rt_critic', 'rt_audience']))

In [7]:
def make_db(d):
    df = pd.DataFrame.from_dict(d, orient='index', columns=['rt_critic', 'rt_audience'])
    df.reset_index(inplace=True)
    df.rename(columns={'index':'movie'}, inplace=True)
    df.to_csv('data/rt.csv', index=False)
    return df

In [8]:
imdb_df = pd.read_csv('data/imdb_meta.csv')

In [9]:
imdb_df

,movie,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...
8887,Falling Overnight,2011,86 min,"Drama, Romance",6.7,40,1002
8888,Fremde Haut,2005,97 min,Drama,7.0,60,1002
8889,Bam gua nat,2008,144 min,Drama,7.1,73,1001
8890,Awful Nice,2013,92 min,Comedy,4.9,42,1000


In [81]:
movies = imdb_df['movie']

In [84]:
dummy = imdb_df.copy()

In [11]:
years = imdb_df['year']

In [90]:
tomato = {}

In [ ]:
for i in range(6642,8893):
            clear_output(wait=True)
            print("i:{}, movie:{}".format(i, movies[i]))
            a = get_rt_scores(movies[i], years[i])
            tomato[movies[i]] = a
            tom_df = make_db(tomato)

## Removing NAN in rt_critics

In [ ]:
imdb = pd.read_csv("data/imdb_meta.csv")
rt = pd.read_csv("data/rt.csv")

In [ ]:
movies = pd.merge(rt, imdb, how = 'inner', on = 'movie')

In [ ]:
movies.to_csv('data/combo.csv', index=False)

In [153]:
combo = pd.read_csv('data/combo.csv')

In [154]:
combo

,movie,rt_critic,rt_audience,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,91.0,98.0,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,94.0,94.0,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,87.0,91.0,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,79.0,96.0,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,92.0,96.0,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...,...,...
8091,Falling Overnight,40.0,64.0,2011,86 min,"Drama, Romance",6.7,40,1002
8092,Fremde Haut,80.0,81.0,2005,97 min,Drama,7.0,60,1002
8093,Bam gua nat,NaN,NaN,2008,144 min,Drama,7.1,73,1001
8094,Awful Nice,22.0,37.0,2013,92 min,Comedy,4.9,42,1000


In [254]:
mask = combo.rt_critic.isna()


In [256]:
new_vars = ['rt_critic','rt_audience']

In [257]:
test2 = combo.copy()

In [258]:
test2

,movie,rt_critic,rt_audience,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,91.0,98.0,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,94.0,94.0,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,87.0,91.0,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,79.0,96.0,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,92.0,96.0,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...,...,...
8091,Falling Overnight,40.0,64.0,2011,86 min,"Drama, Romance",6.7,40,1002
8092,Fremde Haut,80.0,81.0,2005,97 min,Drama,7.0,60,1002
8093,Bam gua nat,NaN,NaN,2008,144 min,Drama,7.1,73,1001
8094,Awful Nice,22.0,37.0,2013,92 min,Comedy,4.9,42,1000


In [259]:
test2.loc[mask, new_vars] = test2[mask].apply(lambda row : get_rt_scores(row['movie'], row['year']), axis=1)

Léon
https://www.rottentomatoes.com/search?search=L%C3%A9on
('Léon', 1994)
None
V for Vendetta
https://www.rottentomatoes.com/search?search=V%20for%20Vendetta
('V for Vendetta', 2005)
https://rottentomatoes.com/m/v_for_vendetta
get_rt_scores
Movie:V for Vendetta,critic:73,audience:90
De Hobbit: Een onverwachte reis
https://www.rottentomatoes.com/search?search=De%20Hobbit%3A%20Een%20onverwachte%20reis
('De Hobbit: Een onverwachte reis', 2012)
https://rottentomatoes.com/m/reisen_til_julestjernen_2013
get_rt_scores
Movie:De Hobbit: Een onverwachte reis,critic:,audience:50
Amélie
https://www.rottentomatoes.com/search?search=Am%C3%A9lie
('Amélie', 2001)
None
Het leven is mooi
https://www.rottentomatoes.com/search?search=Het%20leven%20is%20mooi
('Het leven is mooi', 1997)
None
Binnenstebuiten
https://www.rottentomatoes.com/search?search=Binnenstebuiten
('Binnenstebuiten', 2015)
None
Gisaengchung
https://www.rottentomatoes.com/search?search=Gisaengchung
('Gisaengchung', 2019)
None
Spider-Man 

None
Scooby-Doo
https://www.rottentomatoes.com/search?search=Scooby-Doo
('Scooby-Doo', 2002)
None
Home
https://www.rottentomatoes.com/search?search=Home
('Home', 2015)
https://rottentomatoes.com/m/771403557
get_rt_scores
Movie:Home,critic:,audience:22
Home
https://www.rottentomatoes.com/search?search=Home
('Home', 2014)
https://rottentomatoes.com/m/home_2015
get_rt_scores
Movie:Home,critic:52,audience:64
The American
https://www.rottentomatoes.com/search?search=The%20American
('The American', 2010)
None
Het regent gehaktballen 2
https://www.rottentomatoes.com/search?search=Het%20regent%20gehaktballen%202
('Het regent gehaktballen 2', 2013)
None
Malèna
https://www.rottentomatoes.com/search?search=Mal%C3%A8na
('Malèna', 2000)
None
The King
https://www.rottentomatoes.com/search?search=The%20King
('The King', 2019)
https://rottentomatoes.com/m/the_king_of_staten_island
get_rt_scores
Movie:The King,critic:75,audience:83
The King
https://www.rottentomatoes.com/search?search=The%20King
('The 

https://rottentomatoes.com/m/critical_choices
get_rt_scores
Movie:Critical Decision,critic:,audience:
Huisdiergeheimen 2
https://www.rottentomatoes.com/search?search=Huisdiergeheimen%202
('Huisdiergeheimen 2', 2019)
https://rottentomatoes.com/m/2_hearts
get_rt_scores
Movie:Huisdiergeheimen 2,critic:17,audience:87
Neko no ongaeshi
https://www.rottentomatoes.com/search?search=Neko%20no%20ongaeshi
('Neko no ongaeshi', 2002)
None
Don Juan DeMarco
https://www.rottentomatoes.com/search?search=Don%20Juan%20DeMarco
('Don Juan DeMarco', 1994)
https://rottentomatoes.com/m/don-juan-demarco
get_rt_scores
Movie:Don Juan DeMarco,critic:69,audience:68
W.
https://www.rottentomatoes.com/search?search=W.
('W.', 2008)
None
Wyatt Earp
https://www.rottentomatoes.com/search?search=Wyatt%20Earp
('Wyatt Earp', 1994)
https://rottentomatoes.com/m/wyatt_earp_return_to_tombstone
get_rt_scores
Movie:Wyatt Earp,critic:,audience:65
The Pirates! In an Adventure with Scientists!
https://www.rottentomatoes.com/search?s

None
Liu lang di qiu
https://www.rottentomatoes.com/search?search=Liu%20lang%20di%20qiu
('Liu lang di qiu', 2019)
https://rottentomatoes.com/m/long_days_journey_into_night
get_rt_scores
Movie:Liu lang di qiu,critic:93,audience:66
Do lok tin si
https://www.rottentomatoes.com/search?search=Do%20lok%20tin%20si
('Do lok tin si', 1995)
None
Son of a Gun
https://www.rottentomatoes.com/search?search=Son%20of%20a%20Gun
('Son of a Gun', 2014)
https://rottentomatoes.com/m/son_of_a_gun_2015
get_rt_scores
Movie:Son of a Gun,critic:62,audience:63
Double Trouble
https://www.rottentomatoes.com/search?search=Double%20Trouble
('Double Trouble', 2009)
None
Samâ uôzu
https://www.rottentomatoes.com/search?search=Sam%C3%A2%20u%C3%B4zu
('Samâ uôzu', 2009)
https://rottentomatoes.com/m/sam_in_the_snow
get_rt_scores
Movie:Samâ uôzu,critic:,audience:
SLC Punk!
https://www.rottentomatoes.com/search?search=SLC%20Punk%21
('SLC Punk!', 1998)
https://rottentomatoes.com/m/slc_punk
get_rt_scores
Movie:SLC Punk!,critic

https://rottentomatoes.com/m/the_human_race_2012
get_rt_scores
Movie:The Human Race,critic:42,audience:58
Happily N'Ever After
https://www.rottentomatoes.com/search?search=Happily%20N%27Ever%20After
("Happily N'Ever After", 2006)
https://rottentomatoes.com/m/happily_never_after
get_rt_scores
Movie:Happily N'Ever After,critic:4,audience:34
One False Move
https://www.rottentomatoes.com/search?search=One%20False%20Move
('One False Move', 1992)
https://rottentomatoes.com/m/one_false_move
get_rt_scores
Movie:One False Move,critic:94,audience:75
Jing wu feng yun: Chen Zhen
https://www.rottentomatoes.com/search?search=Jing%20wu%20feng%20yun%3A%20Chen%20Zhen
('Jing wu feng yun: Chen Zhen', 2010)
None
Nurse 3-D
https://www.rottentomatoes.com/search?search=Nurse%203-D
('Nurse 3-D', 2013)
None
The Crime Boss
https://www.rottentomatoes.com/search?search=The%20Crime%20Boss
('The Crime Boss', 2020)
https://rottentomatoes.com/m/encrypted_the_cyber_crime
get_rt_scores
Movie:The Crime Boss,critic:,audi

...

get_rt_scores
Movie:End of the Spear,critic:41,audience:82
El autor
https://www.rottentomatoes.com/search?search=El%20autor
('El autor', 2017)
https://rottentomatoes.com/m/the_car_road_to_revenge
get_rt_scores
Movie:El autor,critic:,audience:24
Danny's Daydream
https://www.rottentomatoes.com/search?search=Danny%27s%20Daydream
("Danny's Daydream", 2003)
https://rottentomatoes.com/m/daydream_obsession_2004
get_rt_scores
Movie:Danny's Daydream,critic:,audience:
Qianxi mànbo
https://www.rottentomatoes.com/search?search=Qianxi%20m%C3%A0nbo
('Qianxi mànbo', 2001)
None
Wassup Rockers
https://www.rottentomatoes.com/search?search=Wassup%20Rockers
('Wassup Rockers', 2005)
https://rottentomatoes.com/m/wassup_rockers_2006
get_rt_scores
Movie:Wassup Rockers,critic:38,audience:64
Mr. and Mrs. Iyer
https://www.rottentomatoes.com/search?search=Mr.%20and%20Mrs.%20Iyer
('Mr. and Mrs. Iyer', 2002)
https://rottentomatoes.com/m/mr_and_mrs_iyer
get_rt_scores
Movie:Mr. and Mrs. Iyer,critic:,audience:92
Ni na

https://rottentomatoes.com/m/breaking_the_girls
get_rt_scores
Movie:Breaking the Girls,critic:10,audience:28
Aloft
https://www.rottentomatoes.com/search?search=Aloft
('Aloft', 2014)
https://rottentomatoes.com/m/aloft
get_rt_scores
Movie:Aloft,critic:17,audience:25
Nadie Sabe Que Estoy Aquí
https://www.rottentomatoes.com/search?search=Nadie%20Sabe%20Que%20Estoy%20Aqu%C3%AD
('Nadie Sabe Que Estoy Aquí', 2020)
https://rottentomatoes.com/m/those_who_remained
get_rt_scores
Movie:Nadie Sabe Que Estoy Aquí,critic:100,audience:67
Shaonian de ni
https://www.rottentomatoes.com/search?search=Shaonian%20de%20ni
('Shaonian de ni', 2019)
None
The Perfect Game
https://www.rottentomatoes.com/search?search=The%20Perfect%20Game
('The Perfect Game', 2009)
https://rottentomatoes.com/m/1197992-perfect_game
get_rt_scores
Movie:The Perfect Game,critic:57,audience:70
Gap tung kei hap
https://www.rottentomatoes.com/search?search=Gap%20tung%20kei%20hap
('Gap tung kei hap', 2014)
None
Bamui haebyun-eoseo honja
h

None
Play the Game
https://www.rottentomatoes.com/search?search=Play%20the%20Game
('Play the Game', 2009)
https://rottentomatoes.com/m/play_the_game
get_rt_scores
Movie:Play the Game,critic:28,audience:61
Ba bai
https://www.rottentomatoes.com/search?search=Ba%20bai
('Ba bai', 2020)
None
Blue Juice
https://www.rottentomatoes.com/search?search=Blue%20Juice
('Blue Juice', 1995)
https://rottentomatoes.com/m/blue_juice
get_rt_scores
Movie:Blue Juice,critic:,audience:56
The Killing Jar
https://www.rottentomatoes.com/search?search=The%20Killing%20Jar
('The Killing Jar', 2010)
https://rottentomatoes.com/m/10012232-killing_jar
get_rt_scores
Movie:The Killing Jar,critic:0,audience:36
Meduzot
https://www.rottentomatoes.com/search?search=Meduzot
('Meduzot', 2007)
None
Aurora
https://www.rottentomatoes.com/search?search=Aurora
('Aurora', 2012)
None
Hon zin 2
https://www.rottentomatoes.com/search?search=Hon%20zin%202
('Hon zin 2', 2016)
None
Pure
https://www.rottentomatoes.com/search?search=Pure
('P

https://rottentomatoes.com/m/the_resurrection_of_gavin_stone
get_rt_scores
Movie:The Resurrection of Gavin Stone,critic:54,audience:84
Go Get Some Rosemary
https://www.rottentomatoes.com/search?search=Go%20Get%20Some%20Rosemary
('Go Get Some Rosemary', 2009)
https://rottentomatoes.com/m/kids_go_to_the_woods_kids_get_dead_2012
get_rt_scores
Movie:Go Get Some Rosemary,critic:,audience:67
Desert Dancer
https://www.rottentomatoes.com/search?search=Desert%20Dancer
('Desert Dancer', 2014)
https://rottentomatoes.com/m/desert_dancer
get_rt_scores
Movie:Desert Dancer,critic:32,audience:62
Bringing Up Bobby
https://www.rottentomatoes.com/search?search=Bringing%20Up%20Bobby
('Bringing Up Bobby', 2011)
None
Aftermath
https://www.rottentomatoes.com/search?search=Aftermath
('Aftermath', 2017)
https://rottentomatoes.com/m/aftermath_2017
get_rt_scores
Movie:Aftermath,critic:42,audience:25
Aftermath
https://www.rottentomatoes.com/search?search=Aftermath
('Aftermath', 2014)
https://rottentomatoes.com/m/

https://rottentomatoes.com/m/1193221-1193221-muhammad_the_last_prophet
get_rt_scores
Movie:Muhammad: The Last Prophet,critic:,audience:76
I Will Follow You Into the Dark
https://www.rottentomatoes.com/search?search=I%20Will%20Follow%20You%20Into%20the%20Dark
('I Will Follow You Into the Dark', 2012)
https://rottentomatoes.com/m/i_will_follow_you_into_the_dark_2012
get_rt_scores
Movie:I Will Follow You Into the Dark,critic:20,audience:20
Mila
https://www.rottentomatoes.com/search?search=Mila
('Mila', 2020)
https://rottentomatoes.com/m/milan_talkies
get_rt_scores
Movie:Mila,critic:,audience:
Donne-moi la main
https://www.rottentomatoes.com/search?search=Donne-moi%20la%20main
('Donne-moi la main', 2008)
https://rottentomatoes.com/m/10008864-shake_hands_with_the_devil
get_rt_scores
Movie:Donne-moi la main,critic:55,audience:88
La ville est tranquille
https://www.rottentomatoes.com/search?search=La%20ville%20est%20tranquille
('La ville est tranquille', 2000)
None
They Remain
https://www.rot

In [263]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8096 entries, 0 to 8095
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        8096 non-null   object 
 1   rt_critic    7926 non-null   object 
 2   rt_audience  7897 non-null   object 
 3   year         8096 non-null   int64  
 4   runtime      8096 non-null   object 
 5   genre        8096 non-null   object 
 6   imdb         8096 non-null   float64
 7   meta         8096 non-null   int64  
 8   imdb_votes   8096 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 569.4+ KB


In [265]:
test2.to_csv('data/movies_data.csv', index=False)

In [ ]:
#movies = pd.merge(rt, imdb, how = 'inner', on = 'movie')